In [1]:
%pip install tensorflow-gpu

In [0]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection  import train_test_split

In [0]:
import pandas as pd
from sklearn.datasets import load_boston

In [0]:
data=load_boston()

In [5]:
print(data)

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]]), 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 1

In [0]:
#Now lets convert this data to dataframe

data_x=pd.DataFrame(data.data,columns=data.feature_names)
data_y=pd.DataFrame(data.target,columns=['target'])

In [7]:
print(data_x.head())
print(data_y.head())

      CRIM    ZN  INDUS  CHAS    NOX  ...  RAD    TAX  PTRATIO       B  LSTAT
0  0.00632  18.0   2.31   0.0  0.538  ...  1.0  296.0     15.3  396.90   4.98
1  0.02731   0.0   7.07   0.0  0.469  ...  2.0  242.0     17.8  396.90   9.14
2  0.02729   0.0   7.07   0.0  0.469  ...  2.0  242.0     17.8  392.83   4.03
3  0.03237   0.0   2.18   0.0  0.458  ...  3.0  222.0     18.7  394.63   2.94
4  0.06905   0.0   2.18   0.0  0.458  ...  3.0  222.0     18.7  396.90   5.33

[5 rows x 13 columns]
   target
0    24.0
1    21.6
2    34.7
3    33.4
4    36.2


In [0]:
#Now let us combine this data

data_final=pd.concat([data_x,data_y],axis=1)

In [9]:
print(data_final.head())

      CRIM    ZN  INDUS  CHAS    NOX  ...    TAX  PTRATIO       B  LSTAT  target
0  0.00632  18.0   2.31   0.0  0.538  ...  296.0     15.3  396.90   4.98    24.0
1  0.02731   0.0   7.07   0.0  0.469  ...  242.0     17.8  396.90   9.14    21.6
2  0.02729   0.0   7.07   0.0  0.469  ...  242.0     17.8  392.83   4.03    34.7
3  0.03237   0.0   2.18   0.0  0.458  ...  222.0     18.7  394.63   2.94    33.4
4  0.06905   0.0   2.18   0.0  0.458  ...  222.0     18.7  396.90   5.33    36.2

[5 rows x 14 columns]


In [10]:
feature_names=data.feature_names
print(feature_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [0]:
feature_columns=[]
for each in feature_names:
  feature_columns.append(tf.feature_column.numeric_column(each))

In [0]:
layer=tf.keras.layers.DenseFeatures(feature_columns)

In [0]:
#Now lets convert the dataframe to dataset 

def dataset(frame,shuffle=True,batch=32):
  dataframe=frame.copy()
  label=dataframe.pop('target')
  ds=tf.data.Dataset.from_tensor_slices((dict(dataframe),label))
  if shuffle:
    ds=ds.shuffle(buffer_size=len(dataframe))
  ds=ds.batch(batch_size=batch)
  return ds

In [0]:
train,test=train_test_split(data_final,test_size=0.2,random_state=1)
train,val=train_test_split(train,test_size=0.2)

In [0]:
train=dataset(train)
test=dataset(test,shuffle=False)
val=dataset(val)

In [0]:
#Model time

model=tf.keras.Sequential([
                         layer,
                         tf.keras.layers.Dense(128,activation='relu'),
                         tf.keras.layers.Dense(128,activation='relu'),
                         tf.keras.layers.Dense(1,activation=None)  
])

model.compile(optimizer='adam',loss='mse',metrics=['mse'])

In [17]:
model.fit(train,validation_data=val,epochs=700)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/700
11/11 [==============================] - 1s 135ms/step - loss: 327.6858 - mse: 340.8539 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00
Epoch 2/700
11/11 [==============================] - 0s 10ms/step - loss: 118.4581 - mse: 117.5728 - val_loss: 73.9423 - val_mse: 78.5500
Epoch 3/700
11/11 [==============================] - 0s 8ms/step - loss: 66.1599 - mse: 65.7375 - val_loss: 68.7829 - val_mse: 63.4491
Epoch 4/700
11/11 [==============================] - 0s 10ms/step - loss: 55.0883 - mse: 5

In [18]:
loss,mse=model.evaluate(test)

print("Test loss is ",loss)
print("Mean Squared Error is ",mse)

4/4 [==============================] - 0s 6ms/step - loss: 14.3771 - mse: 15.9270
Test loss is  14.377061367034912
Mean Squared Error is  15.92705
